# Week 5 Quiz

This notebook contains the SQL Quiz for Week 5. Section 1 uses the New York City data we know and love. Section 2 uses building footprint data from the Google Open Buildings dataset.

INSTRUCTIONS:

Run this notebook in Google Colab. The answer to each quesiton will be a number or a string. Input these into the corresponding question on Moodle. You have 90 minutes to attempt the quiz, so if you get stuck on a question, move on.

Make sure you run all of the cells of code in order, especially the ones that already have code in them! If you run into serious problems, try clicking on the "runtime" tab above and selecting "restart session and run all".

In [1]:
%pip install duckdb duckdb-engine jupysql leafmap

Note: you may need to restart the kernel to use updated packages.


In [3]:
import duckdb
import leafmap
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [4]:
%%sql
duckdb:///:memory:
INSTALL spatial;
LOAD spatial;


,Success


# Section 1



The code below downloads the data on New York City that we've been working with so far.

In [4]:
url='https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip'
leafmap.download_file(url, unzip=True)

Downloading...
From: https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip
To: /Users/ollieballinger/Google Drive/Work/UCL/Teaching/CASA0025 BSABD/CASA0025/notebooks/quiz/postgis-workshop-2020.zip
100%|██████████| 22.5M/22.5M [00:30<00:00, 748kB/s] 


Extracting files...


'/Users/ollieballinger/Google Drive/Work/UCL/Teaching/CASA0025 BSABD/CASA0025/notebooks/quiz/postgis-workshop-2020.zip'

The following line of code creates a table called `nyc_neighborhoods` from a shapefile called `nyc_neighborhoods.shp`, located in the `postgis-workshop/data/` folder.

In [5]:
%%sql

CREATE TABLE nyc_neighborhoods AS SELECT * FROM "postgis-workshop/data/nyc_neighborhoods.shp";

,Success


The `%%sql` at the top of the cell allows you to run SQL code in the rest of the cell:

In [6]:
%%sql

SELECT * FROM nyc_neighborhoods;

,BORONAME,NAME,geom
0,Brooklyn,Bensonhurst,"[2, 4, 33, 0, 0, 0, 0, 0, 54, 71, 14, 73, 198,..."
1,Manhattan,East Village,"[2, 4, 144, 0, 0, 0, 0, 0, 35, 215, 14, 73, 13..."
2,Manhattan,West Village,"[2, 4, 83, 0, 0, 0, 0, 0, 161, 95, 14, 73, 212..."
3,The Bronx,Throggs Neck,"[2, 4, 133, 0, 0, 0, 0, 0, 128, 232, 17, 73, 1..."
4,The Bronx,Wakefield-Williamsbridge,"[2, 4, 118, 0, 0, 0, 0, 0, 83, 85, 17, 73, 17,..."
...,...,...,...
124,Brooklyn,Red Hook,"[5, 4, 56, 0, 0, 0, 0, 0, 18, 0, 14, 73, 149, ..."
125,Queens,Douglastown-Little Neck,"[2, 4, 236, 0, 0, 0, 0, 0, 251, 165, 19, 73, 7..."
126,Queens,Whitestone,"[5, 4, 236, 0, 0, 0, 0, 0, 17, 75, 18, 73, 86,..."
127,Queens,Steinway,"[5, 4, 63, 0, 0, 0, 0, 0, 124, 87, 16, 73, 87,..."


## Question 1

Create the following tables using the corresponding shapefiles.
- nyc_census_blocks
- nyc_homicides
- nyc_streets
- nyc_subway_stations

In [7]:
%%sql
CREATE TABLE nyc_streets AS SELECT * FROM "postgis-workshop/data/nyc_streets.shp";
CREATE TABLE nyc_subway_stations AS SELECT * FROM "postgis-workshop/data/nyc_subway_stations.shp";
CREATE TABLE nyc_census_blocks AS SELECT * FROM "postgis-workshop/data/nyc_census_blocks.shp";
CREATE TABLE nyc_homicides AS SELECT * FROM "postgis-workshop/data/nyc_homicides.shp";

: 

## Question 2:
How many rows are there in the nyc_homicides table?



In [8]:
%%sql
SELECT count(*) FROM nyc_homicides;

,count_star()
0,3983


## Question 3:

How many homicides were there in Brooklyn in 2008?

In [9]:
%%sql
SELECT count(*)
FROM nyc_homicides
WHERE BORONAME = 'Brooklyn'
AND YEAR = 2008

,count_star()
0,206


## Question 4:

Which neighborhood of New York had the most murders in 2010?

In [10]:
%%sql
SELECT n.NAME as neighborhood, count(*) as murders
FROM nyc_homicides as h
JOIN nyc_neighborhoods as n
ON ST_WITHIN(h.geom, n.geom)
WHERE h.YEAR = 2010
GROUP BY neighborhood
ORDER BY murders DESC


,neighborhood,murders
0,Bedford-Stuyvesant,30
1,South Bronx,21
2,Harlem,14
3,East Brooklyn,13
4,Jamaica,10
...,...,...
62,Westerleigh-Castleton,1
63,Bensonhurst,1
64,Midland Beach,1
65,Ardon Heights,1


## Question 5

Calculate the per-capita murder rates for each borough of New York.

In [11]:
%%sql
SELECT c.BORONAME as borough, count(h.*)/SUM(c.POPN_TOTAL) as murders_pc
FROM nyc_homicides as h
JOIN nyc_census_blocks as c
ON ST_WITHIN(h.geom, c.geom)
GROUP BY borough
ORDER BY murders_pc DESC

,borough,murders_pc
0,Staten Island,0.003534
1,Queens,0.003199
2,Brooklyn,0.001781
3,The Bronx,0.001562
4,Manhattan,0.001339


# Section 2

This section uses building footprint data generated by Google. Each row is a polygon that specifies the outline of a building detected using AI and satellite imagery.

![](https://sites.research.google/open-buildings/static/img/buildings-header-light-cropped.png)

We're going to be working with all of the building footprints identified in Kigali, the capital city of Rwanda.

The code below creates a table called `kigali_buildings` by reading a parquet file hosted a the URL in the final line.

In [13]:
%%sql

CREATE TABLE kigali_buildings AS
SELECT full_plus_code as building_id, ST_GeomFromWKB(geometry) as geom
FROM read_parquet('https://data.source.coop/cholmes/google-open-buildings/v2/geoparquet-admin1/country=RWA/City_of_Kigali.parquet');

,Success


Our table has two columns; one called `building_id` which is a unique identifier for each building footprint, and `geom` which contains the geometry information.

In [14]:
%%sql
select * from kigali_buildings

,admin_1,building_id,confidence,geom
0,City of Kigali,6G9FWXPV+VQRP,0.6472,"[2, 4, 96, 0, 0, 0, 0, 0, 127, 244, 239, 65, 1..."
1,City of Kigali,6G9FWXPV+WP9F,0.6473,"[2, 4, 96, 0, 0, 0, 0, 0, 100, 244, 239, 65, 2..."
2,City of Kigali,6G9FWXPV+WPV9,0.6338,"[2, 4, 96, 0, 0, 0, 0, 0, 77, 244, 239, 65, 14..."
3,City of Kigali,6G9FWXPV+WQ3R,0.7050,"[2, 4, 96, 0, 0, 0, 0, 0, 129, 244, 239, 65, 1..."
4,City of Kigali,6G9FWXPV+WQWC,0.7966,"[2, 4, 96, 0, 0, 0, 0, 0, 132, 244, 239, 65, 0..."
...,...,...,...,...
786870,City of Kigali,6GCG6553+2XP4,0.6250,"[2, 4, 128, 0, 0, 0, 0, 0, 79, 61, 241, 65, 22..."
786871,City of Kigali,6GCG6553+2XQ9,0.6284,"[2, 4, 96, 0, 0, 0, 0, 0, 98, 61, 241, 65, 212..."
786872,City of Kigali,6GCG6556+26H3,0.7305,"[2, 4, 96, 0, 0, 0, 0, 0, 224, 72, 241, 65, 17..."
786873,City of Kigali,6GCG6556+27F9,0.8145,"[2, 4, 128, 0, 0, 0, 0, 0, 250, 72, 241, 65, 6..."


## Question 6
Complete the code below to create a column called `area` which calculates the area of each building using the `geom` column and multiply the result by 12356260000 to get the area in meters.

In [15]:
%%sql
ALTER TABLE kigali_buildings ADD COLUMN area DOUBLE;
UPDATE kigali_buildings SET area =  ST_Area(geom)*12356260000


,Success


## Question 7

How many buildings in Kigali are larger than 100 square meters?

In [16]:
%%sql
SELECT COUNT(*)
FROM kigali_buildings
WHERE area>100;

,count_star()
0,101937


## Question 8

How many square meters is the largest building in Kigali?

In [17]:
%%sql
SELECT building_id, area
FROM kigali_buildings
ORDER BY area DESC

,building_id,area
0,6GCG43HP+PJH7,6703.286316
1,6GCG3572+QXJ4,6697.154950
2,6GCG24C5+4HHC,5982.208029
3,6GCG23HJ+6FQ7,5960.072035
4,6GCG257Q+MH85,5096.519344
...,...,...
786870,6GCG22RX+P6XW,2.267773
786871,6GCG22GV+6V8P,2.250158
786872,6GCG537H+488J,2.196751
786873,6GCG34Q7+XQ69,2.191241


## Question 9

How many buildings are within a distance of 0.001 from the largest building in Kigali? (be careful)

In [18]:
%%sql
SELECT COUNT(*)
FROM kigali_buildings AS a
JOIN kigali_buildings AS b
ON ST_DWithin(a.geom, b.geom, 0.001)
AND a.building_id != b.building_id
AND a.area >6703

,count_star()
0,7


## Question 10

A common characteristic of informal settlements (sometimes referred to as "slums") is the presence of a large number of buildings in close proximity to each other. In this question, we are going to create a measure of building density.

First, let's create a new table called `sample` which contains the first 1000 rows of data in our datset:

In [19]:
%%sql

CREATE TABLE sample AS SELECT * FROM kigali_buildings limit 1000;

,Success


Spatially join the `sample` table to itself to identify buildings within a distance of 0.001 of each other. (Hint: `count` and `group by` will be useful functions).

What is the `building_id` of the building with the greatest number of buildings within a distance of 0.001?

In [21]:
%%sql

SELECT a.building_id, COUNT(*) as count
FROM sample AS a
JOIN sample AS b
ON ST_DWithin(a.geom, b.geom, 0.001)
AND a.building_id <> b.building_id
GROUP BY a.building_id
ORDER BY count DESC


,building_id,count
0,6G9FXX2V+Q8HP,95
1,6G9FWXVQ+CWQ5,94
2,6G9FXX2V+M8CJ,94
3,6G9FXX2V+M8MR,94
4,6G9FXX2V+Q92V,94
...,...,...
995,6G9FWXWW+7CRW,3
996,6G9FWXWW+8996,3
997,6G9FWXXV+6QMW,2
998,6G9FWXXV+5M4X,2
